# Образец результата

прогноз на апрель - август 2023, сделанный на данных до марта включительно

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os
os.chdir(os.path.dirname(os.getcwd()))

from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
fact = pd.read_csv("D:/Oracul/data/fact_train_test.csv", sep=";", decimal=",", encoding="windows-1251")
fact["period"] = fact["period"].astype("datetime64[ns]")

In [3]:
# Факт на задание
fact = pd.read_csv("D:/Oracul/data/fact_train_test.csv", sep=";", decimal=",", encoding="windows-1251")
fact["period"] = fact["period"].astype("datetime64[ns]")
# fact - уже обрезанная выборка

# Пример прогноза - наивный, копия последнего факта
result = []
last_known_fact_month = fact["period"].max()
test_periods = pd.date_range(start=last_known_fact_month, periods=6, freq='MS', inclusive='right')
for period in test_periods:
    print(period)
    month_forecast = fact[fact["period"] == last_known_fact_month].reset_index(drop=True)
    month_forecast["period"] = period
    result.append(month_forecast)
result = pd.concat(result).reset_index(drop=True)
result.rename(columns={"real_wagon_count": "forecast_wagon_count", "real_weight": "forecast_weight"}, inplace=True)
result.to_csv("D:/Oracul/data/forecast_example.csv", index=False, sep=";", decimal=",", encoding="windows-1251")


2023-04-01 00:00:00
2023-05-01 00:00:00
2023-06-01 00:00:00
2023-07-01 00:00:00
2023-08-01 00:00:00


In [4]:
client_map = pd.read_csv("D:/Oracul/data/client_mapping.csv", sep=";", decimal=",", encoding="windows-1251")
station_map = pd.read_csv("D:/Oracul/data/station_mapping.csv", sep=";", decimal=",", encoding="windows-1251")
freight_map = pd.read_csv("D:/Oracul/data/freight_mapping.csv", sep=";", decimal=",", encoding="windows-1251")

In [5]:
fact["month"] = fact["period"].dt.month
result["month"] = result["period"].astype("datetime64[ns]").dt.month

In [6]:
fact = fact.merge(client_map, on=['client_sap_id'], how='left')

In [7]:
station_map_sendr = station_map.rename(columns= {'station_id' : 'sender_station_id', 'department_name' : 'sender_department_name', 'railway_name' : 'sender_railway_name'})

In [8]:
fact = fact.merge(station_map_sendr, on=['sender_station_id'], how='left')

In [9]:
station_map_resip = station_map.rename(columns= {'station_id' : 'recipient_station_id', 'department_name' : 'recipient_department_name', 'railway_name' : 'recipient_railway_name'})

In [10]:
fact = fact.merge(station_map_resip, on=['recipient_station_id'], how='left')

In [11]:
fact_train = fact[fact["period"] < "2022-04-01"]
fact_test = fact[(fact["period"] >= "2022-04-01") & (fact["period"] < "2022-09-01")]

In [12]:
fact_train

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count,month,holding_name,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name
0,2012-07-01,1,5,1,328,1193,30252,13005,10036,71.0,1,7,1423.0,153,12,75,4
1,2012-10-01,1,5,1,328,1193,30252,11376,10036,210.0,3,10,1423.0,153,12,99,23
2,2014-03-01,0,1,1,328,3472,30252,29548,10036,67.0,1,3,1423.0,153,12,16,11
3,2014-03-01,0,1,1,328,3472,30252,29158,10036,67.0,1,3,1423.0,153,12,118,11
4,2014-03-01,0,2,1,328,3472,30252,27484,10036,66.0,1,3,1423.0,153,12,94,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027199,2022-03-01,1,5,2,690,1180,23677,33107,26824,396.0,6,3,2234.0,123,31,43,29
3027200,2022-03-01,1,5,2,690,1180,23677,32975,26824,194.0,3,3,2234.0,123,31,103,29
3027201,2022-03-01,1,5,2,690,1180,23677,33307,26824,201.0,3,3,2234.0,123,31,150,29
3027202,2022-03-01,1,5,2,690,1180,23677,20404,26824,1638.0,25,3,2234.0,123,31,17,39


In [13]:
fact_train_6M = fact_train[fact_train["period"] > "2021-09-01"]

In [14]:
holding6 = fact_train_6M[["sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count", "holding_name"]].groupby(["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"]).mean().rename(columns={"real_wagon_count" : "holding6M"})

In [15]:
fact_train_12M = fact_train[fact_train["period"] > "2021-03-01"]

In [16]:
holding12 = fact_train_12M[["sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count", "holding_name"]].groupby(["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"]).mean().rename(columns={"real_wagon_count" : "holding12M"})

In [17]:
month_mean = fact_train[["month", "sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"]).mean().rename(columns={"real_wagon_count" : "month_mean"})
mean = fact_train[["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"]).mean().rename(columns={"real_wagon_count" : "mean"})
month_last = fact_train_12M[["month", "sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"]).mean().rename(columns={"real_wagon_count" : "month_last"})

In [18]:
fact_test = fact_test.merge(month_mean, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"], how='left')
fact_test = fact_test.merge(month_last, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"], how='left')
fact_test = fact_test.merge(mean, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"], how='left')
fact_test = fact_test.merge(holding6, on = ["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"], how='left')
fact_test = fact_test.merge(holding12, on = ["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"], how='left')

In [19]:
fact_test = fact_test.replace(np.nan, 0)

In [20]:
fact_train_6M = fact[(fact["period"] >= "2021-10-01") & (fact["period"] < "2022-04-01")]
mean6M = fact_train_6M[["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"]).mean().rename(columns={"real_wagon_count" : "mean6M"})

In [21]:
fact_test = fact_test.merge(mean6M, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"], how='left')

In [22]:
fact_test = fact_test.replace(np.nan, 0)

In [23]:
fact_train_12M = fact[(fact["period"] >= "2021-04-01") & (fact["period"] < "2022-04-01")]
mean12M = fact_train_12M[["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"]).mean().rename(columns={"real_wagon_count" : "mean12M"})

In [24]:
fact_test = fact_test.merge(mean12M, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"], how='left')

In [25]:
fact_test

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,...,sender_railway_name,recipient_department_name,recipient_railway_name,month_mean,month_last,mean,holding6M,holding12M,mean6M,mean12M
0,2022-04-01,0,0,0,-1,349,38725,38966,26331,1655.0,...,6,23,6,0.0,0.0,10.000000,0.0,0.000000,10.000000,10.000000
1,2022-04-01,0,0,0,-1,349,38760,38966,26331,273.0,...,6,23,6,0.0,0.0,2.000000,0.0,0.000000,2.000000,2.000000
2,2022-04-01,0,0,0,-1,1863,12660,35196,27478,43.0,...,4,9,10,0.0,0.0,1.000000,0.0,0.000000,1.000000,1.000000
3,2022-04-01,0,0,0,-1,1863,11291,30536,27585,0.0,...,23,59,12,0.0,0.0,0.000000,0.0,0.000000,0.000000,NaN
4,2022-04-01,0,0,0,-1,2039,13440,13537,26597,14.0,...,30,140,30,0.0,0.0,0.142857,0.0,0.000000,0.142857,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221497,2022-08-01,1,5,2,690,1180,23677,33107,26824,448.0,...,31,43,29,0.0,0.0,7.000000,9.0,6.136364,7.000000,7.000000
221498,2022-08-01,1,5,2,690,1180,23677,32975,26824,198.0,...,31,103,29,0.0,0.0,3.000000,3.0,2.125000,3.000000,3.000000
221499,2022-08-01,1,5,2,690,1180,23677,20404,26824,1383.0,...,31,17,39,0.0,0.0,14.500000,12.6,12.600000,14.500000,14.500000
221500,2022-08-01,1,5,2,690,1180,23677,34400,26824,63.0,...,31,160,43,0.0,0.0,9.000000,6.0,5.777778,9.000000,9.000000


In [26]:
X = fact_test[["month", "mean", "month_mean", "rps", "podrod", "filial", "mean6M", "mean12M", "holding6M", "holding12M", "month_last"]]
y = fact_test["real_wagon_count"]

In [27]:
from catboost import CatBoostRegressor

In [28]:
model = CatBoostRegressor(n_estimators=300)

In [29]:
model.fit(X, y)

Learning rate set to 0.25577
0:	learn: 117.2088030	total: 173ms	remaining: 51.8s
1:	learn: 107.9777141	total: 188ms	remaining: 28s
2:	learn: 102.2505718	total: 202ms	remaining: 20s
3:	learn: 98.8721645	total: 217ms	remaining: 16s
4:	learn: 96.3981488	total: 230ms	remaining: 13.6s
5:	learn: 94.7071998	total: 244ms	remaining: 12s
6:	learn: 93.6327179	total: 257ms	remaining: 10.8s
7:	learn: 92.9739833	total: 271ms	remaining: 9.89s
8:	learn: 92.0575420	total: 286ms	remaining: 9.23s
9:	learn: 90.8873467	total: 300ms	remaining: 8.69s
10:	learn: 90.6491935	total: 313ms	remaining: 8.22s
11:	learn: 90.3491672	total: 326ms	remaining: 7.82s
12:	learn: 90.1344241	total: 338ms	remaining: 7.47s
13:	learn: 89.4952514	total: 353ms	remaining: 7.21s
14:	learn: 89.3337861	total: 368ms	remaining: 7s
15:	learn: 89.1585231	total: 381ms	remaining: 6.77s
16:	learn: 89.0968382	total: 397ms	remaining: 6.61s
17:	learn: 89.0268140	total: 413ms	remaining: 6.46s
18:	learn: 88.8523454	total: 429ms	remaining: 6.34s
1

In [30]:
preds = [round(max(i, 0)) for i in model.predict(X)]

In [31]:
from sklearn.metrics import mean_absolute_error

In [32]:
mean_absolute_error(preds, y)

14.846339987900786

In [33]:
np.mean(y)

20.86448429359554

In [34]:
result = result.merge(client_map, on=['client_sap_id'], how='left')

In [35]:
result = result.replace(np.nan, 0)

In [36]:
fact_6M = fact[(fact["period"] >= "2022-09-01")]
fact_12M = fact[(fact["period"] >= "2022-03-01")]

In [37]:
month_mean = fact[["month", "sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"]).mean().rename(columns={"real_wagon_count" : "month_mean"})
mean = fact[["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"]).mean().rename(columns={"real_wagon_count" : "mean"})
month_last = fact_12M[["month", "sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"]).mean().rename(columns={"real_wagon_count" : "month_last"})
mean6M = fact_6M[["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"]).mean().rename(columns={"real_wagon_count" : "mean6M"})
holding6 = fact_6M[["sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count", "holding_name"]].groupby(["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"]).mean().rename(columns={"real_wagon_count" : "holding6M"})
holding12 = fact_12M[["sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count", "holding_name"]].groupby(["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"]).mean().rename(columns={"real_wagon_count" : "holding12M"})
mean12M = fact_12M[["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "real_wagon_count"]].groupby(["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"]).mean().rename(columns={"real_wagon_count" : "mean12M"})

In [38]:
result = result.merge(mean6M, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"], how='left')
result = result.merge(mean, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"], how='left')
result = result.merge(month_mean, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"], how='left')
result = result.merge(month_last, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps", "month"], how='left')
result = result.merge(mean12M, on = ["sender_organisation_id", "sender_station_id", "recipient_station_id", "podrod", "rps"], how='left')
result = result.merge(holding6, on = ["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"], how='left')
result = result.merge(holding12, on = ["sender_station_id", "recipient_station_id", "podrod", "rps", "holding_name"], how='left')

In [39]:
result = result.replace(np.nan, 0)

In [40]:
preds = [round(max(i, 0)) for i in model.predict(result[["month", "mean", "month_mean", "rps", "podrod", "filial", "mean6M", "mean12M", "holding6M", "holding12M", "month_last"]])]

In [41]:
#result["forecast_wagon_count"] = np.round(np.array(preds) * 0.2 + np.array(result["forecast_wagon_count"]) * 0.8)
result["forecast_wagon_count"] = np.round(np.array(preds))
result = result[["period", "rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id", "forecast_weight", "forecast_wagon_count"]]
result

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,forecast_weight,forecast_wagon_count
0,2023-04-01,0,0,0,2275,3291,23976,26423,27983,73.0,0
1,2023-04-01,0,0,0,2275,3294,23976,26423,27983,146.0,0
2,2023-04-01,0,0,0,2275,3294,23976,24516,27983,876.0,3
3,2023-04-01,0,0,0,-1,349,38725,38966,27437,473.0,0
4,2023-04-01,0,0,0,-1,349,38754,38966,26664,63.0,0
...,...,...,...,...,...,...,...,...,...,...,...
239600,2023-08-01,1,5,2,1346,1482,31438,31482,0,69.0,3
239601,2023-08-01,1,5,2,1346,1492,31438,36091,27275,70.0,2
239602,2023-08-01,1,5,2,1346,1492,31438,35450,27275,70.0,0
239603,2023-08-01,1,5,2,1346,1492,31438,31482,0,207.0,3


In [42]:
result.to_csv("D:/Oracul/data/forecast_example.csv", index=False, sep=";", decimal=",", encoding="windows-1251")

# Оценка результата

In [43]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os
os.chdir(os.path.dirname(os.getcwd()))

import pandas as pd
import numpy as np


In [43]:
def add_master_data_mappings(df: pd.DataFrame) -> pd.DataFrame:
    # = Пути к справочникам - откорректировать если в реальной системе будут лежать по другому адресу =
    client_mapping_file = "D:/Oracul/data/client_mapping.csv"
    freight_mapping_file = "D:/Oracul/data/freight_mapping.csv"
    station_mapping_file = "D:/Oracul/data/station_mapping.csv"

    # Клиент - холдинг
    client_mapping = pd.read_csv(
        client_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(df, client_mapping, how="left", on="client_sap_id")

    # Груз
    freight_mapping = pd.read_csv(
        freight_mapping_file, sep=";", decimal=",", encoding="windows-1251"
    )
    df = pd.merge(df, freight_mapping, how="left", on="freight_id")

    # Станции
    station_mapping = pd.read_csv(
        station_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("sender_"),
        how="left",
        on="sender_station_id",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("recipient_"),
        how="left",
        on="recipient_station_id",
    )

    return df


def evaluate(fact: pd.DataFrame, forecast: pd.DataFrame, public: bool = True) -> float:
    # = Параметры для расчета метрики =
    accuracy_granularity = [
        "period",
        "rps",
        "holding_name",
        "sender_department_name",
        "recipient_department_name",
    ]
    fact_value, forecast_value = "real_wagon_count", "forecast_wagon_count"
    if public:
        metric_weight = np.array([0.0, 1.0, 0.0, 0.0, 0.0])
    else:
        metric_weight = np.array([0.1, 0.6, 0.1, 0.1, 0.1])

    # = Собственно расчет метрик =
    # 1. Добавляем сущности верхних уровней гранулярности по справочникам
    fact = add_master_data_mappings(fact)
    forecast = add_master_data_mappings(forecast)

    # 2. Расчет KPI
    compare_data = pd.merge(
        fact.groupby(accuracy_granularity, as_index=False)[fact_value].sum(),
        forecast.groupby(accuracy_granularity, as_index=False)[forecast_value].sum(),
        how="outer",
        on=accuracy_granularity,
    ).fillna(0)
    # Против самых хитрых - нецелочисленный прогноз вагоноотправок не принимаем
    compare_data[fact_value] = np.around(compare_data[fact_value]).astype(int)
    compare_data[forecast_value] = np.around(compare_data[forecast_value]).astype(int)

    # 3. Рассчитаем метрики для каждого месяца в выборке
    compare_data["ABS_ERR"] = abs(
        compare_data[forecast_value] - compare_data[fact_value]
    )
    compare_data["MAX"] = abs(compare_data[[forecast_value, fact_value]].max(axis=1))
    summary = compare_data.groupby("period")[
        [forecast_value, fact_value, "ABS_ERR", "MAX"]
    ].sum()
    summary["Forecast Accuracy"] = 1 - summary["ABS_ERR"] / summary["MAX"]

    # 4. Взвесим метрики отдельных месяцев для получения одной цифры score
    score = (
        summary["Forecast Accuracy"].sort_index(ascending=True) * metric_weight
    ).sum()

    return score


def calc_score_public(fact: pd.DataFrame, forecast: pd.DataFrame) -> float:
    return evaluate(fact, forecast, public=True)


def calc_score_private(fact: pd.DataFrame, forecast: pd.DataFrame) -> float:
    return evaluate(fact, forecast, public=False)



In [44]:
# = Примеры файлов для проверки =
validation_file = "D:/Oracul/data/fact_train_test.csv"
forecast_file = "D:/Oracul/data/forecast_example.csv"

# Валидационный датасет
fact = pd.read_csv(validation_file, sep=";", decimal=",", encoding="windows-1251")
# print("Валидационный датасет:", fact.shape)
# Прогноз
forecast = pd.read_csv(forecast_file, sep=";", decimal=",", encoding="windows-1251")
# print("Прогноз:", forecast.shape)
add_master_data_mappings
# Скорим
score_public = calc_score_public(fact, forecast)
score_private = calc_score_private(fact, forecast)
print(f"Public score: {score_public}")
print(f"Private score: {score_private}")


ValueError: operands could not be broadcast together with shapes (90,) (5,) 